# References

https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-1-autoencoder-d9a5f8795af4  
https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-2-exploring-latent-space-with-adversarial-2d53a6f8a4f9


In [1]:
import tensorflow as tf
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
from matplotlib import gridspec
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Get the MNIST data
mnist = input_data.read_data_sets('./Data', one_hot=True)

# Parameters
input_dim = 784
n_l1 = 1000
n_l2 = 1000
z_dim = 2
batch_size = 100
n_epochs = 1000
learning_rate = 0.001
beta1 = 0.9
results_path = ""


# Placeholders for input data and the targets
x_input = tf.placeholder(dtype=tf.float32, shape=[batch_size, input_dim], name='Input')
x_target = tf.placeholder(dtype=tf.float32, shape=[batch_size, input_dim], name='Target')
decoder_input = tf.placeholder(dtype=tf.float32, shape=[1, z_dim], name='Decoder_input')


def generate_image_grid(sess, op):
    """
    Generates a grid of images by passing a set of numbers to the decoder and getting its output.
    :param sess: Tensorflow Session required to get the decoder output
    :param op: Operation that needs to be called inorder to get the decoder output
    :return: None, displays a matplotlib window with all the merged images.
    """
    x_points = np.arange(0, 1, 1.5).astype(np.float32)
    y_points = np.arange(0, 1, 1.5).astype(np.float32)

    nx, ny = len(x_points), len(y_points)
    plt.subplot()
    gs = gridspec.GridSpec(nx, ny, hspace=0.05, wspace=0.05)

    for i, g in enumerate(gs):
        z = np.concatenate(([x_points[int(i / ny)]], [y_points[int(i % nx)]]))
        z = np.reshape(z, (1, 2))
        x = sess.run(op, feed_dict={decoder_input: z})
        ax = plt.subplot(g)
        img = np.array(x.tolist()).reshape(28, 28)
        ax.imshow(img, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('auto')
    plt.show()


def form_results():
    """
    Forms folders for each run to store the tensorboard files, saved models and the log files.
    :return: three string pointing to tensorboard, saved models and log paths respectively.
    """
    #folder_name = "/{0}_{1}_{2}_{3}_{4}_{5}_autoencoder". \
     #   format(datetime.datetime.now(), z_dim, learning_rate, batch_size, n_epochs, beta1)
    folder_name = "test_autoenc1"
    tensorboard_path = results_path + folder_name + '/Tensorboard'
    saved_model_path = results_path + folder_name + '/Saved_models/'
    log_path = results_path + folder_name + '/log'
    if not os.path.exists(results_path + folder_name):
        os.mkdir(results_path + folder_name)
        os.mkdir(tensorboard_path)
        os.mkdir(saved_model_path)
        os.mkdir(log_path)
    return tensorboard_path, saved_model_path, log_path


def dense(x, n1, n2, name):
    """
    Used to create a dense layer.
    :param x: input tensor to the dense layer
    :param n1: no. of input neurons
    :param n2: no. of output neurons
    :param name: name of the entire dense layer.i.e, variable scope name.
    :return: tensor with shape [batch_size, n2]
    """
    with tf.variable_scope(name, reuse=None):
        weights = tf.get_variable("weights", shape=[n1, n2],
                                  initializer=tf.random_normal_initializer(mean=0., stddev=0.01))
        bias = tf.get_variable("bias", shape=[n2], initializer=tf.constant_initializer(0.0))
        out = tf.add(tf.matmul(x, weights), bias, name='matmul')
        return out


# The autoencoder network
def encoder(x, reuse=False):
    """
    Encode part of the autoencoder
    :param x: input to the autoencoder
    :param reuse: True -> Reuse the encoder variables, False -> Create or search of variables before creating
    :return: tensor which is the hidden latent variable of the autoencoder.
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Encoder'):
        e_dense_1 = tf.nn.relu(dense(x, input_dim, n_l1, 'e_dense_1'))
        e_dense_2 = tf.nn.relu(dense(e_dense_1, n_l1, n_l2, 'e_dense_2'))
        latent_variable = dense(e_dense_2, n_l2, z_dim, 'e_latent_variable')
        return latent_variable


def decoder(x, reuse=False):
    """
    Decoder part of the autoencoder
    :param x: input to the decoder
    :param reuse: True -> Reuse the decoder variables, False -> Create or search of variables before creating
    :return: tensor which should ideally be the input given to the encoder.
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Decoder'):
        d_dense_1 = tf.nn.relu(dense(x, z_dim, n_l2, 'd_dense_1'))
        d_dense_2 = tf.nn.relu(dense(d_dense_1, n_l2, n_l1, 'd_dense_2'))
        output = tf.nn.sigmoid(dense(d_dense_2, n_l1, input_dim, 'd_output'))
        return output


def train(train_model):
    """
    Used to train the autoencoder by passing in the necessary inputs.
    :param train_model: True -> Train the model, False -> Load the latest trained model and show the image grid.
    :return: does not return anything
    """
    with tf.variable_scope(tf.get_variable_scope()):
        encoder_output = encoder(x_input)
        decoder_output = decoder(encoder_output)

    with tf.variable_scope(tf.get_variable_scope()):
        decoder_image = decoder(decoder_input, reuse=True)

    # Loss
    loss = tf.reduce_mean(tf.square(x_target - decoder_output))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1).minimize(loss)
    init = tf.global_variables_initializer()

    # Visualization
    tf.summary.scalar(name='Loss', tensor=loss)
    tf.summary.histogram(name='Encoder Distribution', values=encoder_output)
    input_images = tf.reshape(x_input, [-1, 28, 28, 1])
    generated_images = tf.reshape(decoder_output, [-1, 28, 28, 1])
    tf.summary.image(name='Input Images', tensor=input_images, max_outputs=10)
    tf.summary.image(name='Generated Images', tensor=generated_images, max_outputs=10)
    summary_op = tf.summary.merge_all()

    # Saving the model
    saver = tf.train.Saver()
    step = 0
    with tf.Session() as sess:
        sess.run(init)
        if train_model:
            tensorboard_path, saved_model_path, log_path = form_results()
            writer = tf.summary.FileWriter(logdir=tensorboard_path, graph=sess.graph)
            for i in range(n_epochs):
                n_batches = int(mnist.train.num_examples / batch_size)
                for b in range(n_batches):
                    batch_x, _ = mnist.train.next_batch(batch_size)
                    sess.run(optimizer, feed_dict={x_input: batch_x, x_target: batch_x})
                    if b % 50 == 0:
                        batch_loss, summary = sess.run([loss, summary_op], feed_dict={x_input: batch_x, x_target: batch_x})
                        writer.add_summary(summary, global_step=step)
                        print("Loss: {}".format(batch_loss))
                        print("Epoch: {}, iteration: {}".format(i, b))
                        with open(log_path + '/log.txt', 'a') as log:
                            log.write("Epoch: {}, iteration: {}\n".format(i, b))
                            log.write("Loss: {}\n".format(batch_loss))
                    step += 1
                saver.save(sess, save_path=saved_model_path, global_step=step)
            print("Model Trained!")
            print("Tensorboard Path: {}".format(tensorboard_path))
            print("Log Path: {}".format(log_path + '/log.txt'))
            print("Saved Model Path: {}".format(saved_model_path))
        else:
            all_results = os.listdir(results_path)
            all_results.sort()
            saver.restore(sess,
                          save_path=tf.train.latest_checkpoint(results_path + '/' + all_results[-1] + '/Saved_models/'))
            generate_image_grid(sess, op=decoder_image)

if __name__ == '__main__':
    train(train_model=True)

Extracting ./Data\train-images-idx3-ubyte.gz
Extracting ./Data\train-labels-idx1-ubyte.gz
Extracting ./Data\t10k-images-idx3-ubyte.gz
Extracting ./Data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Summary name Encoder Distribution is illegal; using Encoder_Distribution instead.
INFO:tensorflow:Summary name Input Images is illegal; using Input_Images instead.
INFO:tensorflow:Summary name Generated Images is illegal; using Generated_Images instead.
Loss: 0.23097795248031616
Epoch: 0, iteration: 0
Loss: 0.07362150400876999
Epoch: 0, iteration: 50
Loss: 0.06351190060377121
Epoch: 0, iteration: 100
Loss: 0.06496652960777283
Epoch: 0, iteration: 150
Loss: 0.05162171274423599
Epoch: 0, iteration: 200
Loss: 0.05311557650566101
Epoch: 0, iteration: 250
Loss: 0.04728425294160843
Epoch: 0, iteration: 300
Loss: 0.04683656617999077
Epoch: 0, iteration: 350
Loss: 0.05288131907582283
Epoch: 0, iteration: 400
Loss: 0.04658329486846924
Epoch: 0, iteration: 450
Loss: 0.04817115142941475
Epoch: 0, iteration

Loss: 0.03451547399163246
Epoch: 13, iteration: 450
Loss: 0.03228411450982094
Epoch: 13, iteration: 500
Loss: 0.03351959213614464
Epoch: 14, iteration: 0
Loss: 0.035536058247089386
Epoch: 14, iteration: 50
Loss: 0.03733286261558533
Epoch: 14, iteration: 100
Loss: 0.034374531358480453
Epoch: 14, iteration: 150
Loss: 0.03507673740386963
Epoch: 14, iteration: 200
Loss: 0.03451656550168991
Epoch: 14, iteration: 250
Loss: 0.03290232643485069
Epoch: 14, iteration: 300
Loss: 0.033729732036590576
Epoch: 14, iteration: 350
Loss: 0.032700102776288986
Epoch: 14, iteration: 400
Loss: 0.03369234874844551
Epoch: 14, iteration: 450
Loss: 0.034910429269075394
Epoch: 14, iteration: 500
Loss: 0.03326960280537605
Epoch: 15, iteration: 0
Loss: 0.03222474083304405
Epoch: 15, iteration: 50
Loss: 0.033714521676301956
Epoch: 15, iteration: 100
Loss: 0.036365676671266556
Epoch: 15, iteration: 150
Loss: 0.03394145146012306
Epoch: 15, iteration: 200
Loss: 0.0326884500682354
Epoch: 15, iteration: 250
Loss: 0.0318

Loss: 0.03432900086045265
Epoch: 28, iteration: 100
Loss: 0.03225371241569519
Epoch: 28, iteration: 150
Loss: 0.03259475156664848
Epoch: 28, iteration: 200
Loss: 0.03088277392089367
Epoch: 28, iteration: 250
Loss: 0.033107101917266846
Epoch: 28, iteration: 300
Loss: 0.03237343952059746
Epoch: 28, iteration: 350
Loss: 0.03211355209350586
Epoch: 28, iteration: 400
Loss: 0.029713887721300125
Epoch: 28, iteration: 450
Loss: 0.03258940577507019
Epoch: 28, iteration: 500
Loss: 0.034849680960178375
Epoch: 29, iteration: 0
Loss: 0.033930569887161255
Epoch: 29, iteration: 50
Loss: 0.03066621720790863
Epoch: 29, iteration: 100
Loss: 0.03639902547001839
Epoch: 29, iteration: 150
Loss: 0.030525967478752136
Epoch: 29, iteration: 200
Loss: 0.03200643137097359
Epoch: 29, iteration: 250
Loss: 0.032977402210235596
Epoch: 29, iteration: 300
Loss: 0.031716883182525635
Epoch: 29, iteration: 350
Loss: 0.02898116409778595
Epoch: 29, iteration: 400
Loss: 0.034477584064006805
Epoch: 29, iteration: 450
Loss: 0

Loss: 0.03403624892234802
Epoch: 42, iteration: 300
Loss: 0.03113735280930996
Epoch: 42, iteration: 350
Loss: 0.03385406360030174
Epoch: 42, iteration: 400
Loss: 0.031148454174399376
Epoch: 42, iteration: 450
Loss: 0.03125477209687233
Epoch: 42, iteration: 500
Loss: 0.030987393110990524
Epoch: 43, iteration: 0
Loss: 0.031038448214530945
Epoch: 43, iteration: 50
Loss: 0.033029407262802124
Epoch: 43, iteration: 100
Loss: 0.03463532030582428
Epoch: 43, iteration: 150
Loss: 0.031538400799036026
Epoch: 43, iteration: 200
Loss: 0.030998624861240387
Epoch: 43, iteration: 250
Loss: 0.031023958697915077
Epoch: 43, iteration: 300
Loss: 0.02859436348080635
Epoch: 43, iteration: 350
Loss: 0.030318796634674072
Epoch: 43, iteration: 400
Loss: 0.03244955092668533
Epoch: 43, iteration: 450
Loss: 0.032679684460163116
Epoch: 43, iteration: 500
Loss: 0.03284607082605362
Epoch: 44, iteration: 0
Loss: 0.03226068615913391
Epoch: 44, iteration: 50
Loss: 0.03320160508155823
Epoch: 44, iteration: 100
Loss: 0.0

Loss: 0.03272153437137604
Epoch: 56, iteration: 450
Loss: 0.0327666774392128
Epoch: 56, iteration: 500
Loss: 0.031238116323947906
Epoch: 57, iteration: 0
Loss: 0.03153255954384804
Epoch: 57, iteration: 50
Loss: 0.031909823417663574
Epoch: 57, iteration: 100
Loss: 0.03131132945418358
Epoch: 57, iteration: 150
Loss: 0.029595088213682175
Epoch: 57, iteration: 200
Loss: 0.02916545234620571
Epoch: 57, iteration: 250
Loss: 0.032989200204610825
Epoch: 57, iteration: 300
Loss: 0.03150827810168266
Epoch: 57, iteration: 350
Loss: 0.02925117313861847
Epoch: 57, iteration: 400
Loss: 0.03404288366436958
Epoch: 57, iteration: 450
Loss: 0.031758327037096024
Epoch: 57, iteration: 500
Loss: 0.03047451376914978
Epoch: 58, iteration: 0
Loss: 0.032242998480796814
Epoch: 58, iteration: 50
Loss: 0.03241145610809326
Epoch: 58, iteration: 100
Loss: 0.030764995142817497
Epoch: 58, iteration: 150
Loss: 0.03313596174120903
Epoch: 58, iteration: 200
Loss: 0.030858296900987625
Epoch: 58, iteration: 250
Loss: 0.031

...

Loss: 0.0267806313931942
Epoch: 253, iteration: 400
Loss: 0.028409237042069435
Epoch: 253, iteration: 450
Loss: 0.028214046731591225
Epoch: 253, iteration: 500
Loss: 0.030100403353571892
Epoch: 254, iteration: 0
Loss: 0.02755686827003956
Epoch: 254, iteration: 50
Loss: 0.02616981603205204
Epoch: 254, iteration: 100
Loss: 0.025715509429574013
Epoch: 254, iteration: 150
Loss: 0.02706209197640419
Epoch: 254, iteration: 200
Loss: 0.02927723526954651
Epoch: 254, iteration: 250
Loss: 0.028379544615745544
Epoch: 254, iteration: 300
Loss: 0.02554306946694851
Epoch: 254, iteration: 350
Loss: 0.026295552030205727
Epoch: 254, iteration: 400
Loss: 0.026944532990455627
Epoch: 254, iteration: 450
Loss: 0.02807975932955742
Epoch: 254, iteration: 500
Loss: 0.027408983558416367
Epoch: 255, iteration: 0
Loss: 0.027357975021004677
Epoch: 255, iteration: 50
Loss: 0.025222353637218475
Epoch: 255, iteration: 100
Loss: 0.026275554671883583
Epoch: 255, iteration: 150
Loss: 0.028515730053186417
Epoch: 255, ite

Loss: 0.0263817235827446
Epoch: 267, iteration: 400
Loss: 0.027433808892965317
Epoch: 267, iteration: 450
Loss: 0.027734767645597458
Epoch: 267, iteration: 500
Loss: 0.02747715450823307
Epoch: 268, iteration: 0
Loss: 0.027959639206528664
Epoch: 268, iteration: 50
Loss: 0.028120186179876328
Epoch: 268, iteration: 100
Loss: 0.025708254426717758
Epoch: 268, iteration: 150
Loss: 0.024606501683592796
Epoch: 268, iteration: 200
Loss: 0.027850359678268433
Epoch: 268, iteration: 250
Loss: 0.02408352494239807
Epoch: 268, iteration: 300
Loss: 0.027156464755535126
Epoch: 268, iteration: 350
Loss: 0.02773948758840561
Epoch: 268, iteration: 400
Loss: 0.026608049869537354
Epoch: 268, iteration: 450
Loss: 0.02876688726246357
Epoch: 268, iteration: 500
Loss: 0.028771262615919113
Epoch: 269, iteration: 0
Loss: 0.02748350240290165
Epoch: 269, iteration: 50
Loss: 0.028316298499703407
Epoch: 269, iteration: 100
Loss: 0.027275776490569115
Epoch: 269, iteration: 150
Loss: 0.026382969692349434
Epoch: 269, it

Loss: 0.028001222759485245
Epoch: 281, iteration: 400
Loss: 0.024310050532221794
Epoch: 281, iteration: 450
Loss: 0.028981074690818787
Epoch: 281, iteration: 500
Loss: 0.02578016184270382
Epoch: 282, iteration: 0
Loss: 0.027255980297923088
Epoch: 282, iteration: 50
Loss: 0.02518637292087078
Epoch: 282, iteration: 100
Loss: 0.029077913612127304
Epoch: 282, iteration: 150
Loss: 0.026633938774466515
Epoch: 282, iteration: 200
Loss: 0.02591247297823429
Epoch: 282, iteration: 250
Loss: 0.027667606249451637
Epoch: 282, iteration: 300
Loss: 0.02877230942249298
Epoch: 282, iteration: 350
Loss: 0.027898963540792465
Epoch: 282, iteration: 400
Loss: 0.026883572340011597
Epoch: 282, iteration: 450
Loss: 0.028020493686199188
Epoch: 282, iteration: 500
Loss: 0.025711745023727417
Epoch: 283, iteration: 0
Loss: 0.026642698794603348
Epoch: 283, iteration: 50
Loss: 0.028686964884400368
Epoch: 283, iteration: 100
Loss: 0.026464980095624924
Epoch: 283, iteration: 150
Loss: 0.025390131399035454
Epoch: 283,

Loss: 0.027374114841222763
Epoch: 295, iteration: 400
Loss: 0.027662863954901695
Epoch: 295, iteration: 450
Loss: 0.03026638738811016
Epoch: 295, iteration: 500
Loss: 0.025801582261919975
Epoch: 296, iteration: 0
Loss: 0.027691781520843506
Epoch: 296, iteration: 50
Loss: 0.0272982157766819
Epoch: 296, iteration: 100
Loss: 0.027459099888801575
Epoch: 296, iteration: 150
Loss: 0.0254385806620121
Epoch: 296, iteration: 200
Loss: 0.026745550334453583
Epoch: 296, iteration: 250
Loss: 0.026328785344958305
Epoch: 296, iteration: 300
Loss: 0.026952890679240227
Epoch: 296, iteration: 350
Loss: 0.028918743133544922
Epoch: 296, iteration: 400
Loss: 0.028821062296628952
Epoch: 296, iteration: 450
Loss: 0.025882475078105927
Epoch: 296, iteration: 500
Loss: 0.027290556579828262
Epoch: 297, iteration: 0
Loss: 0.02833101898431778
Epoch: 297, iteration: 50
Loss: 0.02596973069012165
Epoch: 297, iteration: 100
Loss: 0.028102360665798187
Epoch: 297, iteration: 150
Loss: 0.028846781700849533
Epoch: 297, it

KeyboardInterrupt: 